In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import base64
import os
import json
import ast
from bs4 import BeautifulSoup

In [3]:
import base64

def base64url_decode(data):
    data = data.encode('utf-8')
    data += b'=' * (4 - (len(data) % 4))
    return base64.urlsafe_b64decode(data)

def is_html_text(text):
    try:
        soup = BeautifulSoup(text, 'html.parser')
        return len(soup.find_all()) > 0
    except:
        return False

def find_text_part(parts):
    res_string = ""
    for part in parts:
        mimeType = part.get('mimeType')
        if mimeType == 'text/plain':
            data = part.get('body', {}).get('data')
            if data:
                text = base64url_decode(data).decode('utf-8')
                if is_html_text(text):
                    soup = BeautifulSoup(text, 'html.parser')
                    res_string = res_string + " " + soup.get_text(separator=' ')
                else:
                    res_string = res_string + " " + text           
        elif mimeType == 'text/html':
            data = part.get('body', {}).get('data')
            if data:
                html = base64url_decode(data).decode('utf-8')
                soup = BeautifulSoup(html, 'html.parser')
                res_string = res_string + " " + soup.get_text(separator=' ')
        elif mimeType == 'multipart/alternative':
            res_string = res_string + " " + find_text_part(part.get('parts', []))
        elif mimeType == 'multipart/related':
            res_string = res_string + " " + find_text_part(part.get('parts', []))
    return res_string

# Function to parse email bodies
def parse_email_body(payload):
    if payload['mimeType'] == 'text/plain':
        data = payload.get('body', {}).get('data')
        if data:
            return base64url_decode(data).decode('utf-8')
    elif payload['mimeType'] == 'text/html':
        data = payload.get('body', {}).get('data')
        if data:
            html = base64url_decode(data).decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
            return soup.get_text(separator=' ')
    elif payload['mimeType'].startswith('multipart/'):
        parts = payload.get('parts', [])
        text = find_text_part(parts)
        if text:
            return text
    return ""

def parse_text(x):
    y = ''.join([i if ord(i) < 128 else ' ' for i in x])
    y = ' '.join(y.strip().split())
    return(y)
    

def get_from(x):
    header_ = x.get("headers")
    return(dict([(elem["name"], elem["value"]) for elem in header_ if elem["name"] in ['Subject', 'From', 'To']]))

In [6]:
df = pd.read_csv("emails_dir/email_30-05-2023-17-27-05_30-05-2023-17-49-21.csv")
df.head()

,id,threadId,labelIds,snippet,payload,sizeEstimate,historyId,internalDate
0,1886c44e3eacd1bd,1886c44e3eacd1bd,"['Label_7781807634347615867', 'CATEGORY_UPDATE...",Senior Quantitative Researcher and other roles...,"{'partId': '', 'mimeType': 'multipart/alternat...",119362,10716728,1685443629000
1,1886c0dde71b424d,1886b089720e7a16,"['UNREAD', 'Label_3265720468738706690', 'CATEG...","Message for Aayush Agrawal Dear Investor, Foli...","{'partId': '', 'mimeType': 'multipart/mixed', ...",373513,10716675,1685439968000
2,1886c01327dbc5f7,1886a9354f28a315,"['UNREAD', 'Label_5479026944618646578', 'CATEG...","Dear AAYUSH, Following is/are the list of tran...","{'partId': '', 'mimeType': 'text/html', 'filen...",7995,10716632,1685438845000
3,1886becb5c7e7f55,1886a9354f28a315,"['UNREAD', 'Label_5479026944618646578', 'CATEG...","Dear AAYUSH, Following is/are the list of tran...","{'partId': '', 'mimeType': 'text/html', 'filen...",7088,10716587,1685437569000
4,1886bdce3bb4c278,1886a9354f28a315,"['UNREAD', 'Label_5479026944618646578', 'CATEG...","Dear AAYUSH, Following is/are the list of tran...","{'partId': '', 'mimeType': 'text/html', 'filen...",7083,10716536,1685436722000


In [7]:
import re
def search_pattern(x, pattern = '(.+?)<'):
    try:
        found = re.search(pattern, x).group(1)
    except AttributeError:
        found = np.nan
    return(found) 

In [8]:
df["body"] = df["payload"].apply(lambda x: parse_text(parse_email_body(ast.literal_eval(x))))
df["headerInfo"] = df["payload"].apply(lambda x:get_from(ast.literal_eval(x)))
df = df.join(pd.json_normalize(df['headerInfo']))
df["fromParsed"] = df["From"].apply(lambda x:search_pattern(str(x), "<(.+?)>"))
df["fromParsed"] = df["fromParsed"].fillna(df["From"])
df.to_csv("emails_dir/emails_parsed.csv")

/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:336: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "http://stat.ethz.ch/education/semesters/ss2014/atsa/Scriptum_v140523.pdf
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:424: MarkupResemblesLocatorWarning: "https://goo.gl/photos/NLzwpxkDF8qRetcs6
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautif